<a href="https://colab.research.google.com/github/CJL-INVEST/invest/blob/main/k_dual_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install bt
!pip install finterstellar
!pip install quantstats
!pip install finance-datareader
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import bt
import finterstellar as fs
import quantstats as qs
import pandas as pd
from datetime import datetime, timedelta
import math
import FinanceDataReader as fdr
import yfinance as yf
%matplotlib inline

In [46]:
ticker = ['360200', '251350','272560', '214980']
ticker_name = ['Kindex미국S&P', 'KODEX 선진국MSCI World','KBSTAR 단기국공채액티브','KODEX단기채권PLUS']
# ticker = ['SPY', 'EFA','AGG', 'BIL']
start_day = datetime(2000,1,1)
end_day = datetime.now()
# end_day = datetime(2019,8,30)

In [47]:
df = pd.DataFrame(columns=ticker_name)

In [48]:
for idx in range(len(ticker)):
  df[ticker_name[idx]] = fdr.DataReader(ticker[idx], start_day - timedelta(days=365), end_day)['Close']

In [49]:
df = df.dropna()

In [50]:
# df =  yf.download(ticker, start=start_day)['Adj Close']
# df = df.dropna()

In [51]:
df

,Kindex미국S&P,KODEX 선진국MSCI World,KBSTAR 단기국공채액티브,KODEX단기채권PLUS
Date,,,,
2020-08-07,9834,15300,101430,102680
2020-08-10,9885,15280,101420,102670
2020-08-11,9900,15360,101425,102670
2020-08-12,9848,15415,101425,102690
2020-08-13,9900,15530,101420,102685
...,...,...,...,...
2022-12-15,13210,20285,103185,104760
2022-12-16,12895,19860,103200,104780
2022-12-19,12745,19555,103220,104800


In [52]:
df = df.resample('M').last()

In [53]:
# 모멘텀 지수 계산 전체 함수

def get_momentum(x):
  temp_list = [0 for i in range(len(x.index))]
  momentum = pd.Series(temp_list, index=x.index)
  try: 
    momentum = df[x.name-timedelta(days=370):x.name-timedelta(days=365)].iloc[-1][ticker]    
  except Exception as e:
    # print('Error: ', str(e))
    pass
  
  return momentum 

In [54]:
# df_mom = df.copy()
# df_mom[ticker] = df[ticker].apply(lambda x: get_momentum(x), axis=1)

In [55]:
# df_mom

In [56]:
df_mon12 = df.pct_change(periods=12)
df_mon12.head(13)

,Kindex미국S&P,KODEX 선진국MSCI World,KBSTAR 단기국공채액티브,KODEX단기채권PLUS
Date,,,,
2020-08-31,NaN,NaN,NaN,NaN
2020-09-30,NaN,NaN,NaN,NaN
2020-10-31,NaN,NaN,NaN,NaN
2020-11-30,NaN,NaN,NaN,NaN
2020-12-31,NaN,NaN,NaN,NaN
2021-01-31,NaN,NaN,NaN,NaN
2021-02-28,NaN,NaN,NaN,NaN
2021-03-31,NaN,NaN,NaN,NaN
2021-04-30,NaN,NaN,NaN,NaN


In [57]:
df_mon12_slected = pd.DataFrame(columns=ticker_name)

In [60]:
def select_item(x):
  asset = None
  temp_list = [False for i in range(len(x.index))]
  selected_info = pd.Series(temp_list, index=x.index)
  if df_mon12[ticker_name[0]].loc[x.name] < df_mon12[ticker_name[3]].loc[x.name]:
      asset = ticker_name[2]
  else:
    if df_mon12[ticker_name[1]].loc[x.name] > df_mon12[ticker_name[0]].loc[x.name]:
      asset = ticker_name[1]
    else:
      asset = ticker_name[0]
  selected_info[asset] = True
  # print('---')
  # print(selected_info)
  return selected_info


In [62]:
df_mon12_slected[ticker_name] = df_mon12[ticker_name].apply(lambda x: select_item(x), axis=1)

In [63]:
df_mon12_slected

,Kindex미국S&P,KODEX 선진국MSCI World,KBSTAR 단기국공채액티브,KODEX단기채권PLUS
Date,,,,
2020-08-31,True,False,False,False
2020-09-30,True,False,False,False
2020-10-31,True,False,False,False
2020-11-30,True,False,False,False
2020-12-31,True,False,False,False
2021-01-31,True,False,False,False
2021-02-28,True,False,False,False
2021-03-31,True,False,False,False
2021-04-30,True,False,False,False


In [64]:
initial_capital = 100000.00

In [65]:
st_original_dual_momentum = bt.Strategy('Original_Dual_Momentum',
                     [
                      bt.algos.RunAfterDate(start_day),
                      bt.algos.RunMonthly(run_on_end_of_period=True),
                      bt.algos.PrintDate(),
                      bt.algos.PrintInfo('{name}:{now}. Value:{_value:0.0f}, Price:{_price:0.4f}'),
                      bt.algos.SelectWhere(df_mon12_slected),
                      bt.algos.WeighEqually(),
                      bt.algos.PrintTempData(),
                      bt.algos.Rebalance()
                     ])

bt_original_dual_momentum = bt.Backtest(st_original_dual_momentum, df, initial_capital=initial_capital)

In [66]:
ret = bt.run(bt_original_dual_momentum)

2020-08-31 00:00:00
Original_Dual_Momentum:2020-08-31 00:00:00. Value:100000, Price:100.0000
{'selected': ['Kindex미국S&P'], 'weights': {'Kindex미국S&P': 1.0}}
2020-09-30 00:00:00
Original_Dual_Momentum:2020-09-30 00:00:00. Value:94663, Price:94.6630
{'selected': ['Kindex미국S&P'], 'weights': {'Kindex미국S&P': 1.0}}
2020-10-31 00:00:00
Original_Dual_Momentum:2020-10-31 00:00:00. Value:88966, Price:88.9660
{'selected': ['Kindex미국S&P'], 'weights': {'Kindex미국S&P': 1.0}}
2020-11-30 00:00:00
Original_Dual_Momentum:2020-11-30 00:00:00. Value:95887, Price:95.8870
{'selected': ['Kindex미국S&P'], 'weights': {'Kindex미국S&P': 1.0}}
2020-12-31 00:00:00
Original_Dual_Momentum:2020-12-31 00:00:00. Value:97786, Price:97.7860
{'selected': ['Kindex미국S&P'], 'weights': {'Kindex미국S&P': 1.0}}
2021-01-31 00:00:00
Original_Dual_Momentum:2021-01-31 00:00:00. Value:100306, Price:100.3060
{'selected': ['Kindex미국S&P'], 'weights': {'Kindex미국S&P': 1.0}}
2021-02-28 00:00:00
Original_Dual_Momentum:2021-02-28 00:00:00. Value:10

In [67]:
ret.display()

Stat                 Original_Dual_Momentum
-------------------  ------------------------
Start                2020-08-30
End                  2022-12-31
Risk-free rate       0.00%

Total Return         22.85%
Daily Sharpe         2.91
Daily Sortino        5.51
CAGR                 9.21%
Max Drawdown         -12.98%
Calmar Ratio         0.71

MTD                  0.27%
3m                   0.37%
6m                   6.98%
YTD                  -6.90%
1Y                   -6.90%
3Y (ann.)            9.21%
5Y (ann.)            -
10Y (ann.)           -
Since Incep. (ann.)  9.21%

Daily Sharpe         2.91
Daily Sortino        5.51
Daily Mean (ann.)    202.93%
Daily Vol (ann.)     69.85%
Daily Skew           -0.13
Daily Kurt           -0.82
Best Day             8.23%
Worst Day            -7.48%

Monthly Sharpe       0.65
Monthly Sortino      1.23
Monthly Mean (ann.)  10.01%
Monthly Vol (ann.)   15.51%
Monthly Skew         -0.15
Monthly Kurt         -0.89
Best Month           8.23%
Worst Mon